# ID별 학대 누적횟수 / 재학대 혐의 여부 / ID별 신고 누적횟수 컬럼 생성

In [1]:
import pandas as pd
import numpy as np

In [47]:
Child_new2= pd.read_csv('./Child_new2.csv',encoding = 'CP949')
Child_new2

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,22,23,26,30,32,45,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_아특_기타.1,아조_원가정보호,아조_가정복귀,아조_사망,아조_분리조치,행조_아동과의분리,행조_고소고발사건처,행조_모니터링,행조_만나지못함,행조_사건처리만나지못함
0,0102010220141200510001,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,0,0,0
1,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,0,1,0,0,0,0,0,1,0,0
2,0402040220141200170001,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,0,0
3,0402040220141200160001,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,0,0
4,0106080420141200200001,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,1,0,0,0,0,0,1,0,0
122031,5115511520180165080001,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,0,1,0,0,0,0,0,1,0,0
122032,6404640420180167080001,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,0,0,0
122033,5715571520180166300001,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,0,0,0


### 학대 혐의 여부 Y인 row만 추출(87487rows)

In [3]:
Child_Y = Child_new2[(Child_new2['판단_학대혐의여부'] == 'Y')]
Child_Y.shape

(87487, 158)

### 학대혐의 Y인 경우의 신고접수일시와 피해아동대상자 dataframe생성 

In [4]:
df2 = Child_Y[['신고_접수일시','피해아동대상자']]

In [5]:
#ID별 정렬 후 신고_접수일시로 오름차순 정렬
#index 89927과 89928 확인
df3 = df2.sort_values(by=["피해아동대상자", "신고_접수일시"], ascending=[True, True])
df3

,신고_접수일시,피해아동대상자
18121,2015-10-26 17:35:00,CASRCP20020110092
12266,2014-04-04 13:51:00,CASRCP20020110126
9734,2014-05-20 11:30:00,CASRCP20020510240
16977,2015-11-17 04:44:00,CASRCP20020610032
28783,2015-03-15 02:20:00,CASRCP20020610138
...,...,...
89058,2018-12-22 15:29:00,CASRCP20190401035
89047,2018-12-22 21:30:00,CASRCP20190401544
88809,2018-12-26 15:00:00,CASRCP20190401747
89927,2018-12-11 18:01:00,CASRCP20200405662


### 신고접수일시와 학대아동ID중복인 경우 삭제

In [6]:
#신고접수일시, 학대아동ID가 똑같은경우 중복 취급(학대행위자만 다른경우로 취급) -> df3에서 중복행 확인 후 삭제 진행
#중복행 확인 
#index89927과 89928 확인
a=df3.duplicated(keep='first')
a[a==True]

39391    True
87825    True
7066     True
17715    True
35906    True
         ... 
89290    True
89293    True
89288    True
89291    True
89928    True
Length: 7400, dtype: bool

In [7]:
#위의 True나온애들 확인(행위자만 다를경우 index는 1차이)
print(df3[df3.index==39390])
print(df3[df3.index==39391])

                   신고_접수일시            피해아동대상자
39390  2016-09-19 15:23:00  CASRCP20020710159
                   신고_접수일시            피해아동대상자
39391  2016-09-19 15:23:00  CASRCP20020710159


In [8]:
#중복 중 첫번째 행만 남기고 삭제 
df3.drop_duplicates(keep='first',inplace=True) #default : keep='first':처음만 남겨두기 

In [9]:
#행 수 감소 : 87487 -> 80087
df3.shape  

(80087, 2)

In [10]:
#중복행 삭제된거 한번더 확인 
a=df3.duplicated(keep='first')
a[a==True]

Series([], dtype: bool)

### New_Abuse_Count (학대누적횟수)컬럼 생성

In [11]:
#New_Abuse_Count변수 생성해서 0으로 일단 채워놓기(defalut값이 0)
df3['New_Abuse_Count'] = 0
df3

,신고_접수일시,피해아동대상자,New_Abuse_Count
18121,2015-10-26 17:35:00,CASRCP20020110092,0
12266,2014-04-04 13:51:00,CASRCP20020110126,0
9734,2014-05-20 11:30:00,CASRCP20020510240,0
16977,2015-11-17 04:44:00,CASRCP20020610032,0
28783,2015-03-15 02:20:00,CASRCP20020610138,0
...,...,...,...
88683,2018-12-27 18:49:00,CASRCP20190306429,0
89058,2018-12-22 15:29:00,CASRCP20190401035,0
89047,2018-12-22 21:30:00,CASRCP20190401544,0
88809,2018-12-26 15:00:00,CASRCP20190401747,0


In [12]:
#피해아동대상자 ID별 누적count
df3['New_Abuse_Count'] = df3.groupby('피해아동대상자').cumcount()

In [13]:
#제대로 만들어졌는지 확인
df3[:50]

,신고_접수일시,피해아동대상자,New_Abuse_Count
18121,2015-10-26 17:35:00,CASRCP20020110092,0
12266,2014-04-04 13:51:00,CASRCP20020110126,0
9734,2014-05-20 11:30:00,CASRCP20020510240,0
16977,2015-11-17 04:44:00,CASRCP20020610032,0
28783,2015-03-15 02:20:00,CASRCP20020610138,0
45766,2016-06-21 13:20:00,CASRCP20020710050,0
39390,2016-09-19 15:23:00,CASRCP20020710159,0
87824,2017-01-11 18:20:00,CASRCP20020710159,1
25739,2015-05-16 15:23:00,CASRCP20020710168,0
39787,2016-09-11 09:15:00,CASRCP20020710206,0


### New_Abuse_Count로 New_재학대혐의여부 컬럼 생성

In [14]:
#df3 index이용 해야 혐의 여부 제대로 들어감 
df3['New_Abuse_Count'].index

Int64Index([18121, 12266,  9734, 16977, 28783, 45766, 39390, 87824, 25739,
            39787,
            ...
            89282, 89281, 89280, 89278, 90165, 88683, 89058, 89047, 88809,
            89927],
           dtype='int64', length=80087)

In [15]:
df3['New_재학대혐의여부'] = np.nan

In [16]:
df3

,신고_접수일시,피해아동대상자,New_Abuse_Count,New_재학대혐의여부
18121,2015-10-26 17:35:00,CASRCP20020110092,0,NaN
12266,2014-04-04 13:51:00,CASRCP20020110126,0,NaN
9734,2014-05-20 11:30:00,CASRCP20020510240,0,NaN
16977,2015-11-17 04:44:00,CASRCP20020610032,0,NaN
28783,2015-03-15 02:20:00,CASRCP20020610138,0,NaN
...,...,...,...,...
88683,2018-12-27 18:49:00,CASRCP20190306429,1,NaN
89058,2018-12-22 15:29:00,CASRCP20190401035,0,NaN
89047,2018-12-22 21:30:00,CASRCP20190401544,0,NaN
88809,2018-12-26 15:00:00,CASRCP20190401747,0,NaN


In [17]:
df3['New_재학대혐의여부'].value_counts()

Series([], Name: New_재학대혐의여부, dtype: int64)

In [18]:
for i in df3['New_Abuse_Count'].index:
    if df3['New_Abuse_Count'][i] >= 1 :
        df3['New_재학대혐의여부'][i] = 'Y'
    else :
        df3['New_재학대혐의여부'][i]= 'N'

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
# 재학대 건수 : 14357 (학대혐의 Y : 87487rows)
df3['New_재학대혐의여부'].value_counts()

N    65730
Y    14357
Name: New_재학대혐의여부, dtype: int64

In [20]:
df3

,신고_접수일시,피해아동대상자,New_Abuse_Count,New_재학대혐의여부
18121,2015-10-26 17:35:00,CASRCP20020110092,0,N
12266,2014-04-04 13:51:00,CASRCP20020110126,0,N
9734,2014-05-20 11:30:00,CASRCP20020510240,0,N
16977,2015-11-17 04:44:00,CASRCP20020610032,0,N
28783,2015-03-15 02:20:00,CASRCP20020610138,0,N
...,...,...,...,...
88683,2018-12-27 18:49:00,CASRCP20190306429,1,Y
89058,2018-12-22 15:29:00,CASRCP20190401035,0,N
89047,2018-12-22 21:30:00,CASRCP20190401544,0,N
88809,2018-12-26 15:00:00,CASRCP20190401747,0,N


In [21]:
#확인 
#pd.set_option('display.max_rows',None)
df3[df3['New_재학대혐의여부']=='Y']

,신고_접수일시,피해아동대상자,New_Abuse_Count,New_재학대혐의여부
87824,2017-01-11 18:20:00,CASRCP20020710159,1,Y
12427,2014-03-31 19:30:00,CASRCP20020810217,1,Y
1747,2014-11-14 23:58:00,CASRCP20021110017,1,Y
57696,2017-12-28 18:40:00,CASRCP20030110016,1,Y
109258,2018-05-19 22:10:00,CASRCP20030110016,2,Y
...,...,...,...,...
89879,2018-12-12 14:00:00,CASRCP20190206517,1,Y
90398,2018-12-05 12:25:00,CASRCP20190302480,1,Y
90399,2018-12-05 12:25:00,CASRCP20190302488,1,Y
88956,2018-12-24 11:35:00,CASRCP20190303152,1,Y


In [22]:
# 재학대 피해아동수 : 11219
a = df3[df3['New_재학대혐의여부']=='Y']
len(a['피해아동대상자'].unique())

11219

### 기존 Child_new2와 합치기

In [23]:
#Child_new2에 index기준으로 New_Abuse_Count 붙이기 - Outer Join
Child_new3 = pd.merge(Child_new2,df3,left_index=True,right_index=True,how="outer")
Child_new3 = Child_new3.drop(['신고_접수일시_y','피해아동대상자_y'],axis=1)
Child_new3.rename(columns = {'신고_접수일시_x':'신고_접수일시','피해아동대상자_x':'피해아동대상자'}, inplace = True)
Child_new3

,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_가정복귀,아조_사망,아조_분리조치,행조_아동과의분리,행조_고소고발사건처,행조_모니터링,행조_만나지못함,행조_사건처리만나지못함,New_Abuse_Count,New_재학대혐의여부
0,0102010220141200510001,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,0,NaN,NaN
1,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,0,0,0,0,0,1,0,0,0.0,N
2,0402040220141200170001,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
3,0402040220141200160001,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
4,0106080420141200200001,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122031,5115511520180165080001,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122032,6404640420180167080001,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
122033,5715571520180166300001,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN


In [24]:
#제대로 들어갔는지 확인
Child_new3.iloc[87824,]

개별사건번호             0904090420170100110002
피해아동대상자                 CASRCP20020710159
학대행위자대상                 CASRCP20020720138
아동_성별                                   F
아동_연령대                             16~17세
                            ...          
행조_모니터링                                 1
행조_만나지못함                                0
행조_사건처리만나지못함                            0
New_Abuse_Count                         1
New_재학대혐의여부                             Y
Name: 87824, Length: 160, dtype: object

In [25]:
Child_new3.columns

Index(['개별사건번호', '피해아동대상자', '학대행위자대상', '아동_성별', '아동_연령대', '아동_내국인여부',
       '아동_최종학력', '아동_직업유형', '아동_거주상태', '아동_친권자유형',
       ...
       '아조_가정복귀', '아조_사망', '아조_분리조치', '행조_아동과의분리', '행조_고소고발사건처', '행조_모니터링',
       '행조_만나지못함', '행조_사건처리만나지못함', 'New_Abuse_Count', 'New_재학대혐의여부'],
      dtype='object', length=160)

In [26]:
#50행만 확인
#pd.set_option('display.max_rows',None)
a = Child_new3[['피해아동대상자','신고_접수일시','New_Abuse_Count','New_재학대혐의여부']]
a = a.sort_values(by=['피해아동대상자','신고_접수일시'], ascending=[True, True])
a.iloc[:50,]

,피해아동대상자,신고_접수일시,New_Abuse_Count,New_재학대혐의여부
18121,CASRCP20020110092,2015-10-26 17:35:00,0.0,N
12266,CASRCP20020110126,2014-04-04 13:51:00,0.0,N
13050,CASRCP20020310027,2014-03-14 10:00:00,NaN,NaN
9734,CASRCP20020510240,2014-05-20 11:30:00,0.0,N
16977,CASRCP20020610032,2015-11-17 04:44:00,0.0,N
28783,CASRCP20020610138,2015-03-15 02:20:00,0.0,N
45766,CASRCP20020710050,2016-06-21 13:20:00,0.0,N
6638,CASRCP20020710151,2014-07-15 10:00:00,NaN,NaN
39390,CASRCP20020710159,2016-09-19 15:23:00,0.0,N
39391,CASRCP20020710159,2016-09-19 15:23:00,NaN,NaN


In [27]:
Child_new3['판단_학대혐의여부'].value_counts()

Y    87487
N    34456
Name: 판단_학대혐의여부, dtype: int64

In [28]:
Child_new3['New_재학대혐의여부'].value_counts()

N    65730
Y    14357
Name: New_재학대혐의여부, dtype: int64

In [29]:
Child_new3.to_csv('c:/Users/SUJIN/Desktop/Child_new3.csv',encoding = 'CP949',index = False)

In [30]:
Child_new3

,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_가정복귀,아조_사망,아조_분리조치,행조_아동과의분리,행조_고소고발사건처,행조_모니터링,행조_만나지못함,행조_사건처리만나지못함,New_Abuse_Count,New_재학대혐의여부
0,0102010220141200510001,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,0,NaN,NaN
1,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,0,0,0,0,0,1,0,0,0.0,N
2,0402040220141200170001,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
3,0402040220141200160001,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
4,0106080420141200200001,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122031,5115511520180165080001,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122032,6404640420180167080001,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
122033,5715571520180166300001,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN


# New_Call_Count(아동ID별 신고횟수) 컬럼 생성 (위 코드 활용)

In [45]:
import pandas as pd
import numpy as np
Child_new3= pd.read_csv('./Child_new3.csv',encoding = 'CP949')
Child_new3

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,22,23,26,30,32,45,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_가정복귀,아조_사망,아조_분리조치,행조_아동과의분리,행조_고소고발사건처,행조_모니터링,행조_만나지못함,행조_사건처리만나지못함,New_Abuse_Count,New_재학대혐의여부
0,0102010220141200510001,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,0,NaN,NaN
1,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,0,0,0,0,0,1,0,0,0.0,N
2,0402040220141200170001,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
3,0402040220141200160001,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,0,NaN,NaN
4,0106080420141200200001,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122031,5115511520180165080001,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,0,0,0,0,0,1,0,0,0.0,N
122032,6404640420180167080001,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN
122033,5715571520180166300001,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,0,NaN,NaN


In [32]:
df2 = Child_new3[['신고_접수일시','피해아동대상자']]

In [33]:
#ID별 정렬 후 신고_접수일시로 오름차순 정렬
df3 = df2.sort_values(by=["피해아동대상자", "신고_접수일시"], ascending=[True, True])
df3

,신고_접수일시,피해아동대상자
18121,2015-10-26 17:35:00,CASRCP20020110092
12266,2014-04-04 13:51:00,CASRCP20020110126
13050,2014-03-14 10:00:00,CASRCP20020310027
9734,2014-05-20 11:30:00,CASRCP20020510240
16977,2015-11-17 04:44:00,CASRCP20020610032
...,...,...
89047,2018-12-22 21:30:00,CASRCP20190401544
88809,2018-12-26 15:00:00,CASRCP20190401747
95630,2018-10-05 11:58:00,CASRCP20190403850
89927,2018-12-11 18:01:00,CASRCP20200405662


### 신고접수일시와 학대아동ID중복인 경우 삭제

In [34]:
#신고접수일시, 학대아동ID가 똑같은경우 중복 취급(학대행위자만 다른경우로 취급) -> df3에서 중복행 확인 후 삭제 진행
#중복행 확인 
a=df3.duplicated(keep='first')
a[a==True]

39391    True
87825    True
7066     True
10577    True
17715    True
         ... 
89289    True
89292    True
89288    True
89291    True
89928    True
Length: 10435, dtype: bool

In [35]:
#중복 중 첫번째 행만 남기고 삭제 
df3.drop_duplicates(keep='first',inplace=True) #default : keep='first':처음만 남겨두기 

In [36]:
#행 수 감소 : 122035 -> 111600
df3.shape

(111600, 2)

In [37]:
#중복행 삭제된거 한번더 확인 
a=df3.duplicated(keep='first')
a[a==True]

Series([], dtype: bool)

### New_Call_Count 만들기

In [38]:
#New_Abuse_Count변수 생성해서 0으로 일단 채워놓기(defalut값이 0)
df3['New_Call_Count'] = 0
df3

,신고_접수일시,피해아동대상자,New_Call_Count
18121,2015-10-26 17:35:00,CASRCP20020110092,0
12266,2014-04-04 13:51:00,CASRCP20020110126,0
13050,2014-03-14 10:00:00,CASRCP20020310027,0
9734,2014-05-20 11:30:00,CASRCP20020510240,0
16977,2015-11-17 04:44:00,CASRCP20020610032,0
...,...,...,...
89058,2018-12-22 15:29:00,CASRCP20190401035,0
89047,2018-12-22 21:30:00,CASRCP20190401544,0
88809,2018-12-26 15:00:00,CASRCP20190401747,0
95630,2018-10-05 11:58:00,CASRCP20190403850,0


In [39]:
#피해아동대상자 ID별 누적count
df3['New_Call_Count'] = df3.groupby('피해아동대상자').cumcount()

In [40]:
#제대로 만들어졌는지 확인
df3[:50]

,신고_접수일시,피해아동대상자,New_Call_Count
18121,2015-10-26 17:35:00,CASRCP20020110092,0
12266,2014-04-04 13:51:00,CASRCP20020110126,0
13050,2014-03-14 10:00:00,CASRCP20020310027,0
9734,2014-05-20 11:30:00,CASRCP20020510240,0
16977,2015-11-17 04:44:00,CASRCP20020610032,0
28783,2015-03-15 02:20:00,CASRCP20020610138,0
45766,2016-06-21 13:20:00,CASRCP20020710050,0
6638,2014-07-15 10:00:00,CASRCP20020710151,0
39390,2016-09-19 15:23:00,CASRCP20020710159,0
87824,2017-01-11 18:20:00,CASRCP20020710159,1


In [41]:
#Child_new3에 index기준으로 New_Abuse_Count 붙이기 - Outer Join
Child_new4 = pd.merge(Child_new3,df3,left_index=True,right_index=True,how="outer")
Child_new4 = Child_new4.drop(['신고_접수일시_y','피해아동대상자_y'],axis=1)
Child_new4.rename(columns = {'신고_접수일시_x':'신고_접수일시','피해아동대상자_x':'피해아동대상자'}, inplace = True)
Child_new4

,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_사망,아조_분리조치,행조_아동과의분리,행조_고소고발사건처,행조_모니터링,행조_만나지못함,행조_사건처리만나지못함,New_Abuse_Count,New_재학대혐의여부,New_Call_Count
0,0102010220141200510001,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
1,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,0,0,0,0,1,0,0,0.0,N,0.0
2,0402040220141200170001,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,NaN,NaN,0.0
3,0402040220141200160001,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,NaN,NaN,0.0
4,0106080420141200200001,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,0,0,0,1,0,0,0.0,N,0.0
122031,5115511520180165080001,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,0,0,0,0,1,0,0,0.0,N,0.0
122032,6404640420180167080001,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
122033,5715571520180166300001,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0


In [42]:
Child_new4['New_Call_Count'].value_counts()

0.0     91559
1.0     14863
2.0      3468
3.0      1057
4.0       372
5.0       152
6.0        64
7.0        30
8.0        15
9.0         8
10.0        5
11.0        3
12.0        2
13.0        1
14.0        1
Name: New_Call_Count, dtype: int64

In [43]:
Child_new4.to_csv('c:/Users/SUJIN/Desktop/Child_new4.csv',encoding = 'CP949',index = False)